### This script is taking all the chapter abstract text, process it, and then call paraphrasing model to perform text based Augmentation. Then save the result as a image(.png) format. Finally, it goes through all the images and perform image transformation and save the result as .png format.

In [1]:
## importing the modules from ETDAugmentation script
from ipynb.fs.full.ETDaugmentation import *
from imgaug import augmenters as iaa
import imgaug as ia
import pandas as pd
import numpy as np
import cv2
import glob
import re

In [2]:
class CHAPTERABS_TEXT_IMG_AUG:
    
    def list_of_phrases(self, phrases_cabs):
        phrases_list = []
        self.phrases_cabs = phrases_cabs
        for text in self.phrases_cabs:
            chapter_abs = parser.preprocess_cabs_abs(text)
            chapter_abs_paraphrase = augmentation.paraphrased_text(chapter_abs)
            chapter_abs_list = [chapter_abs_paraphrase]
            phrases_list.append(chapter_abs_list)
        
        return phrases_list

    ## This function is taking all the augmented text and wrapping it so that it can fit into the page
    ## it also adds heading to the top of the page

    def list_of_chapter_abs(self, chapter_abs_text):
        list_chapter_abs = []
        counter = 0
        self.chapter_abs_text = chapter_abs_text
        for text in self.chapter_abs_text:
            counter += 1
            for row in text:
                chapter_abs_text_wrap = augmentation.wrap_text(row)
                chapter_abs_title = "\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tChapter {}\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tAbstract\n".format(counter) + chapter_abs_text_wrap
                list_chapter_abs.append(chapter_abs_title)
        return list_chapter_abs
    
    def text_on_img(self, fileName, text, size):
        self.fileName = fileName
        self.text = text
        self.size = size
        self.W = 2360
        self.H = 3200
        font = ImageFont.truetype('NimbusMonoPS-Regular.otf', size)
        image = Image.new(mode = "RGB", size = (self.W, self.H), color = "white")
        draw = ImageDraw.Draw(image)
        w, h = draw.textsize(text)
        ## Adjust the texual position in a page and draw the text on a image
        '''
        dedication -- 14 width and height 8; 
        ack -- 15 width and 6 height; 
        gabs -- 13 width and height 16
        abstract -- 16 width and height 12
        chapter abstract -- 18 width and height 6
        '''
        draw.text(((self.W-w)/18,(self.H-h)/6), text, font=font, fill=(0,0,0), spacing=60)
        image.save(fileName)
        
        return image
    
    ## This function is calling text_on_image function and save the image to the directory
    def chapter_abs_aug(self, chapter_abs_text_title):
        self.chapter_abs_text_title = chapter_abs_text_title
        for n, row in enumerate(self.chapter_abs_text_title):
            fileName = ("/home/mchou001/Label-ChapterAbstract/aug_text{}.png".format(n))
            save_image = self.text_on_img(fileName, row, 39) ## dedication -- 43, ack -- 40, chapter abstract - 39 and abstract - 40, gabs -- 38
        
        return save_image
    
    def img_transform(self, img_files):
        self.img_files = img_files
        
        img_augmentation = iaa.Sequential([
            iaa.Affine(rotate=(-6, 6)),
            iaa.AdditiveGaussianNoise(scale=(10, 50)),
            iaa.SaltAndPepper(p=0.1),
            iaa.GaussianBlur(sigma=0.5),
            iaa.LinearContrast(alpha=1),
            iaa.PerspectiveTransform(scale=0.025, keep_size=True)
        ], random_order = True)
        
        lower = 0
        upper = 1000
        while lower < upper:
            for n, images in enumerate(self.img_files[0:]):
                aug_image = img_augmentation(image=cv2.imread(images))
                cv2.imwrite('./Label-ChapterAbstract/504/aug{}.png'.format(lower),aug_image)
                lower += 1

In [3]:
if __name__ == "__main__":
    
    parser = Preprocessor()
    augmentation = Augmentation()
    
    cabs_text = CHAPTERABS_TEXT_IMG_AUG()
    
    etd_data = pd.read_csv('ETD_aug.csv')
    etd_data.set_index("class", inplace = True)

    label_cabs = etd_data.loc["Label-ChapterAbstract"]
    phrases_cabs = label_cabs['text']
    
    cabs = cabs_text.list_of_phrases(phrases_cabs)
    
    cabs_ = cabs_text.list_of_chapter_abs(cabs)
    
    cabs_text_aug = cabs_text.chapter_abs_aug(cabs_)
    
    img_path = sorted(glob.glob('./Label-Dedication/*.png'), key = parser.numericalSort)
    
    augmented_img = cabs_text.img_transform(img_path)
    
    %%time 
    [ x**2 for x in range(10000)]

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 21.7 µs
